In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

df = pd.read_csv('Sao_Paulo_State_of_Sao_Paulo.csv', delimiter=",", encoding = 'cp1252')

df.head()

,Ref,Name,Category,Address,Days,Hours,Number of Reviews,Rating,Excellent,Very Good,Average,Poor,Terrible,Certified
0,/Attraction_Review-g303631-d311966-Reviews-Par...,Parque Ibirapuera,Natureza e parques,"Av. Pedro Álvares Cabral | Vila Mariana, São P...",Dom - Sáb,05:00 - 00:00,36.534,"4,5",65%,29%,4%,1%,1%,True
1,/Attraction_Review-g303631-d550339-Reviews-Pau...,Avenida Paulista,Pontos turísticos e de interesse,"Av. Paulista, São Paulo, Estado de São Paulo 0...",NaN,NaN,29.104,"4,5",62%,29%,7%,1%,1%,True
2,/Attraction_Review-g303631-d4377562-Reviews-Co...,Arena Corinthians,Pontos turísticos e de interesse,"Avenida Miguel Ignacio Curi, 111 | Itaquera, S...",Dom - Sáb,09:30 - 16:30,4.423,"4,5",81%,14%,3%,1%,1%,True
3,/Attraction_Review-g303631-d2349915-Reviews-Al...,Allianz Parque,Pontos turísticos e de interesse,Avenida Francisco Matarazzo 1705 | Água Branca...,Dom - Sáb,10:00 - 17:00,5.138,"4,5",74%,21%,3%,1%,1%,True
4,/Attraction_Review-g303631-d1407969-Reviews-Mu...,Museu do Futebol,Museus,"Praça Charles Miller, S/N | Estádio do Pacaemb...",Ter - Dom,09:00 - 17:00,9.404,"4,5",61%,32%,5%,1%,1%,True


## Pré processamento

### Remoção de atributos irrelevantes

Inicialmente o atributo Ref foi eliminado, pois este possuia utilidade durante o processo de coleta de dados pois era utilizado para obter informações individuais sobre as atrações nas suas respectivas páginas no TripAdvisor, não tendo uso prático para a análise dos dados.

In [2]:
df = df.drop(columns = ['Ref'])

### Tratamento de dados ausentes
Verificar quais atributos possuem entradas com valores ausentes.

In [3]:
df.isnull().sum()

Name                   0
Category              15
Address                0
Days                 467
Hours                467
Number of Reviews      0
Rating                 0
Excellent              0
Very Good              0
Average                0
Poor                   0
Terrible               0
Certified              0
dtype: int64

Alguns pontos não possuem categoria, nesse caso foi optado por substituir o dado ausente por um valor indicando a falta de categoria.

In [4]:
df['Category'].fillna("Sem categoria", inplace = True)

### Tratamento de dados duplicados

In [5]:
df = df.drop_duplicates(['Name', 'Address'])

## Conversão de atributos não numéricos

One hot encoding para o atributo 'Category'

In [5]:
oh = pd.get_dummies(df['Category'])

df = df.drop(columns = ['Category'])

df = df.join(oh)

Obtenção dos tipos dos atributos da base de dados.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 83 columns):
Name                                644 non-null object
Address                             644 non-null object
Days                                177 non-null object
Hours                               177 non-null object
Number of Reviews                   644 non-null float64
Rating                              644 non-null float64
Excellent                           644 non-null float64
Very Good                           644 non-null float64
Average                             644 non-null float64
Poor                                644 non-null float64
Terrible                            644 non-null float64
Certified                           644 non-null bool
Antiquários                         644 non-null uint8
Arenas e estádios                   644 non-null uint8
Atividades ao ar livre              644 non-null uint8
Autódromos                          644 non-null uint

Por algum motivo o TripAdvisor usa o caractere ',' para marcar o ponto decimal na nota das avaliações, porém como as notas são estão entre 0,0 e 5,0 espaçados por 0,5 é algo simples fazer a transformação desses dados.

In [7]:
print(df['Rating'].unique())

['4,5' '4,0' '5,0' '3,5' '3,0' '2,5' '2,0' '1,5' '1,0']


In [8]:
norm_ratings = {
    '5,0': 1.0,
    '4,5': 0.9,
    '4,0': 0.8,
    '3,5': 0.7,
    '3,0': 0.6,
    '2,5': 0.5,
    '2,0': 0.4,
    '1,5': 0.3,
    '1,0': 0.2,
    '0,5': 0.1,
    '0,0': 0.0
}

df['Rating'] = df['Rating'].apply(lambda rating: norm_ratings[rating])

No caso da distribuição das avaliações o valores estão no formato XX% sendo também necessário uma transformação (nesse caso para algo entre 0 e 1).

In [10]:
df['Excellent'] = df['Excellent'].apply(lambda exc: float(int(exc.replace('%', '')) / 100))
df['Very Good'] = df['Very Good'].apply(lambda vg: float(int(vg.replace('%', '')) / 100))
df['Average'] = df['Average'].apply(lambda avg: float(int(avg.replace('%', '')) / 100))
df['Poor'] = df['Poor'].apply(lambda poor: float(int(poor.replace('%', '')) / 100))
df['Terrible'] = df['Terrible'].apply(lambda ter: float(int(ter.replace('%', '')) / 100))

## Usando K-NN pra (tentar) inferir se uma atração é certificada ou não

In [13]:
import matplotlib.pyplot as plt
from sklearn import cluster, neighbors, svm, metrics, preprocessing

In [12]:
df = df.drop(columns = ['Name', 'Address', 'Days', 'Hours'])

In [20]:
trainSample = df.sample(frac = 0.5, random_state = 1) # 80 / 20

testSample = pd.concat([df, trainSample]).drop_duplicates(keep=False)
trainTarget = trainSample["Certified"]
testTarget = testSample["Certified"]


del testSample["Certified"]
del trainSample["Certified"]

In [21]:
knn = neighbors.KNeighborsClassifier(n_neighbors= 3, algorithm="auto") #k vizinhos mais próximos = n_neighbors
knn.fit(trainSample, trainTarget)
resultKNN = knn.predict(testSample)

In [22]:
print("\nK-NN")
print("Acurácia: %0.2f" % (metrics.accuracy_score(testTarget, resultKNN)))


matrizConfusao = metrics.confusion_matrix(testTarget, resultKNN)
print("Matriz de Confusão:\n",matrizConfusao)


K-NN
Acurácia: 0.87
Matriz de Confusão:
 [[237  15]
 [ 24  25]]
